In [11]:
import pandas as pd
import numpy as np
import cloudpickle
import pickle
import time
import dask.dataframe as dskdf
import dask
gl = globals()
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
現在のデータ作成流れ：

■マスタ
    成約データ
■地域
    国勢調査
    都道府県別保有台数
    カスタマDB
    市区町村別保有台数（未受領）
    問い合わせ件数
■車種
    問い合わせ件数
    お気に入り件数
    カタログ
■マスタへのマージ
↓
市区町村名と対応した辞書作成

In [ ]:
あるべきデータ作成流れ：
・マスタ
    ●成約データ（マスタ）
・地域（市区町村名、都道府県名をキーとする）
    ●国勢調査（地域）
    ●都道府県別保有台数（地域）
    ●カスタマDB（地域）
    ▲市区町村別保有台数（未受領、地域）
・車種（車種名をキーとする）
    ●問い合わせ件数（地域、車種）
    ●お気に入り件数（車種）
    ▲カタログ（車種）
↓
・市区町村名と対応した辞書作成
・車種名と対応した辞書作成


In [ ]:
class make_raw_data():

In [2]:
#面倒な前処理部分は今回エクセル上で実施済み
#別途前処理をするコードが必要
import pandas as pd
import numpy as np
import pickle
import time
import dask.dataframe as dskdf
import dask
gl = globals()

<b>--------------------国勢調査ー読み込みー開始-------------------------<b/>

In [41]:
#国勢調査
kokusei = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/kokuseichosa_2015_for_merge.csv")
#全てを英字変換する必要あり
kokusei.columns=["pref_code", "pref_city_code", "capital_city_flg", "city_categories", "chiho", "city_name",
                 "population", "h22_kumikae_population", "population_changed_from_h22_to_h27", "rate_population_changed_from_h22_to_h27",
                 "gross_area", "population_density", "ave_age_from_h22_to_h27", "med_age_from_h22_to_h27", "population",
                 "population_under_15old", "population_from_15_to_64_old", "population_over_65old", "rate_population_under_15_year_old",
                 "rate_population_from_15_to_64_old", "rate_population_over_65old", "population_male","population_male_under_15old",
                 "population_male_from_15_to_64_old", "population_male_over_65old", "rate_population_male_under_15old",
                 "rate_population_male_from_15_to_64_old", "rate_population_male_over_65old", "population_female",
                 "population_female_under_15old", "population_female_from_15_to_64_old", "population_female_over_65old",
                 "rate_population_female_under_15old", "rate_population_female_from_15_to_64_old", "rate_population_female_over_65old",
                 "rate_population_male_to female", "population_jap", "population_no_jap", "num_of_family", "num_of_normal_family",
                 "num_of_family_in_facilities", "h22_kumikae_family", "num_of_normal_family", "num_of_nuclear_family",
                 "num_of_family_no_children", "num_of_family_with_children", "num_of_father_and_children", "num_of_mother_and_children",
                 "num_of_singles", "num_of_over_65old_singles", "num_of_old_married_couples_no_children", "num_of_3generation_family",]


In [42]:
#保有台数
hoyu = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/2015_hoyu_daisu.csv",encoding="utf-8")
hoyu.columns = ["chiho", "num_of_cars_all", "num_of_cars_kei", "num_of_cars_normal", "num_of_familiy", "cars_per_person",
                "normal_cars_per_person", "kei_cars_per_person", "kei_rate", "normal_rate", "kei_share_in_JP", "normal_share_in_JP",
                "guess_2nd_hand_kei_car_sales_per_year", "guess_2nd_hand_normal_car_sales_per_year", "guess_2nd_hand_car_all_sales_per_year"]


In [43]:
#国勢調査と保有台数のマージ
def left_merge_on_chiho(left,right,out):
    out = pd.merge(left,right,on="chiho",how="left")
    #一番下に謎の欠損行が存在するので削除する
    out = out.dropna(subset=['pref_code'])
    return out
koku_hoyu = left_merge_on_chiho(kokusei,hoyu,"koku_hoyu")
koku_hoyu.head()

,pref_code,pref_city_code,capital_city_flg,city_categories,chiho,city_name,population,h22_kumikae_population,population_changed_from_h22_to_h27,rate_population_changed_from_h22_to_h27,gross_area,population_density,ave_age_from_h22_to_h27,med_age_from_h22_to_h27,population,population_under_15old,population_from_15_to_64_old,population_over_65old,rate_population_under_15_year_old,rate_population_from_15_to_64_old,rate_population_over_65old,population_male,population_male_under_15old,population_male_from_15_to_64_old,population_male_over_65old,rate_population_male_under_15old,rate_population_male_from_15_to_64_old,rate_population_male_over_65old,population_female,population_female_under_15old,population_female_from_15_to_64_old,population_female_over_65old,rate_population_female_under_15old,rate_population_female_from_15_to_64_old,rate_population_female_over_65old,rate_population_male_to female,population_jap,population_no_jap,num_of_family,num_of_normal_family,num_of_family_in_facilities,h22_kumikae_family,num_of_normal_family,num_of_nuclear_family,num_of_family_no_children,num_of_family_with_children,num_of_father_and_children,num_of_mother_and_children,num_of_singles,num_of_over_65old_singles,num_of_old_married_couples_no_children,num_of_3generation_family,num_of_cars_all,num_of_cars_kei,num_of_cars_normal,num_of_familiy,cars_per_person,normal_cars_per_person,kei_cars_per_person,kei_rate,normal_rate,kei_share_in_JP,normal_share_in_JP,guess_2nd_hand_kei_car_sales_per_year,guess_2nd_hand_normal_car_sales_per_year,guess_2nd_hand_car_all_sales_per_year
0,1.0,1100.0,1.0,1.0,北海道,札幌市,"1,952,356","1,913,545","38,811",2.0,"1,121.26","1,741.2",46.2,46.4,"1,952,356","221,013","1,235,516","483,534",11.4,63.7,24.9,"910,614","112,743","590,111","201,313",12.5,65.3,22.3,"1,041,742","108,270","645,405","282,221",10.5,62.3,27.2,87.4,"1,937,785","8,820","921,837","920,415","1,422","885,848","920,415","493,644","194,424","213,793","10,049","75,378","375,242","104,650","98,660","20,124","2,805,265",1142062.0,"1,663,203","2,734,906",1.025726,0.608139,0.417587,40.71%,59.29%,3.77%,5.39%,"37,693","161,743","199,436"
1,1.0,1101.0,NaN,0.0,北海道,札幌市中央区,"237,627","220,189","17,438",7.9,46.42,"5,119.1",45.0,44.2,"237,627","23,997","160,339","50,536",10.2,68.3,21.5,"107,756","12,187","74,227","19,750",11.5,69.9,18.6,"129,871","11,810","86,112","30,786",9.2,66.9,23.9,83.0,"235,280","1,891","132,006","131,819",187,"120,741","131,819","52,695","22,332","21,118",878,"8,367","73,164","14,403","9,542","1,221","2,805,265",1142062.0,"1,663,203","2,734,906",1.025726,0.608139,0.417587,40.71%,59.29%,3.77%,5.39%,"37,693","161,743","199,436"
2,1.0,1102.0,NaN,0.0,北海道,札幌市北区,"285,321","278,781","6,540",2.3,63.57,"4,488.3",45.6,45.8,"285,321","32,916","178,950","68,960",11.7,63.7,24.6,"134,534","16,722","86,767","28,866",12.6,65.6,21.8,"150,787","16,194","92,183","40,094",10.9,62.1,27.0,89.2,"281,851","2,023","133,662","133,477",185,"127,440","133,477","70,121","26,738","31,744","1,414","10,225","55,247","14,688","14,221","3,164","2,805,265",1142062.0,"1,663,203","2,734,906",1.025726,0.608139,0.417587,40.71%,59.29%,3.77%,5.39%,"37,693","161,743","199,436"
3,1.0,1103.0,NaN,0.0,北海道,札幌市東区,"261,912","255,873","6,039",2.4,56.97,"4,597.4",45.3,45.0,"261,912","30,716","168,302","61,448",11.8,64.6,23.6,"124,649","15,594","82,491","25,708",12.6,66.6,20.8,"137,263","15,122","85,811","35,740",11.1,62.8,26.2,90.8,"259,619","1,121","124,425","124,251",174,"118,939","124,251","65,606","24,770","28,880","1,401","10,555","51,879","14,050","12,156","2,504","2,805,265",1142062.0,"1,663,203","2,734,906",1.025726,0.608139,0.417587,40.71%,59.29%,3.77%,5.39%,"37,693","161,743","199,436"
4,1.0,1104.0,NaN,0.0,北海道,札幌市白石区,"209,584","204,259","5,325",2.6,34.47,"6,080.2",45.3,44.8,"209,584","23,590","136,879","47,879",11.3,65.7,23.0,"99,020","12,099","66,499","19,954",12.3,67.5,20.2,"110,564","11,491","70,380","27,925",10.5,64.1,25.4,89.6,"207,888",595,"104,608","104,464",144,"100,444","104,464","51,43

<b>
-----------------------国勢調査と保有台数のマージー終わり-----------------------<br>
-----------------------保有車両数マージー始まり-----------------------
<b/>

<b>
-----------------------保有車両数マージー終わり-----------------------<br>
-----------------------問い合わせ件数処理ー始まり-----------------------<br>
<b/>

In [48]:
#以下車データの読み込み
#車両データは全体取れてないので、取った後に縦結合して使う
ku = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_1.csv",
                 delimiter="\t", dtype="object", encoding="cp932",nrows=10000)
ru = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_2.csv",
                 delimiter="\t", dtype="object", encoding="cp932",nrows=10000)
ma = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_3.csv",
                 delimiter="\t", dtype="object", encoding="cp932",nrows=10000)
kuru = pd.concat([ku,ru],axis=0)
kuruma = pd.concat([kuru,ma],axis=0)
#表記の統一
kuruma["b.shikuchoson_jusho_kj"] = kuruma["b.shikuchoson_jusho_kj"].str.replace('ヶ', 'ケ')
#前後スペースの削除
kuruma["b.todofuken_jusho_kj"] = kuruma["b.todofuken_jusho_kj"].map(lambda x: str(x).strip())
# 都道府県欠損を削除
kuruma = kuruma.dropna(subset=["b.todofuken_jusho_kj"])
# 都道府県名に異常値が入っているものを除く
# '〓','null','nan'が入っていたら削除
kuruma = kuruma.ix[kuruma['b.todofuken_jusho_kj'] != "〓"]
kuruma = kuruma.ix[kuruma['b.todofuken_jusho_kj'] != "null"]
kuruma = kuruma.ix[kuruma['b.todofuken_jusho_kj'] != "nan"]

In [49]:
#県単位、市区町村単位での地方別集計（問い合わせ）
#入ってるレベル感が違う(市区町村で終わるものもあれば、町大字まで入っているものもある)
#→市区町村が最後の文字になるように調整した変数を作成する
#市区町村マスタと一致する部分のみ残す
dl = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/市区町村マスタ.csv",names=["pref","city"])

#h市区町村表記揺れを消したい
dl["city"] = dl["city"].str.replace('ヶ', 'ケ')
pre = dl["pref"].unique() #array
#県内市区町村数のカウント用
dl["count"] =1
num = dl.groupby("pref").sum()
dc = num.to_dict()
dic = dc["count"]

biglis = []
c=0
#都道府県別市区町村の格納
for i in pre:
    tmplis = []
    for k in range(int(dic[i])):
#             exec("tmplis.append(dl['city']["+str(c)+"])")
        tmplis.append(dl['city'][c])
        c +=1
    #元データの入り方が札幌市→札幌市〇〇区、といった順で入っているので、上から部分一致で合致を探す際の障害になる
    #ゆえに最初の要素を最後に持ってくる処理が必要
    push_to_last = tmplis.pop(0)
    tmplis.append(push_to_last)
    biglis.append(tmplis)

#辞書の作成
dicts = {key: value for (key, value) in zip(pre,biglis)}

#dfの各要素について、辞書のどこに一致するかを見て、ループで部分一致を探す



In [50]:
ken = kuruma["b.todofuken_jusho_kj"].values
aa = kuruma["b.shikuchoson_jusho_kj"].values
aa
bb = []
counter =[]
#市区町村を格納してる順番が良くないので、並び替えが必要
for k,i in zip(ken,aa):
    count=0
    for l in dicts[k]:
        if l in i:
            bb.append(l)
            counter.append(1)
            break
        count+=1
        if count >=(len(dicts[k])):
            bb.append("")
            counter.append(0)
            break
#概ねOKだけど、県名の後に直接郡の名前が来るケースに対応できていない
# →修正面倒なので後回し
#スクレイピングで持ってくる必要ある：　http://www.geocities.jp/je2kcr/current_allgun.htm
#bbが欠損のものはとりあえず除外検討

#bbをkurumaにマージ
kuruma["city_name"] = bb
kuruma["count"] = 1
#toiawase_dateの型を日付型に変更する必要ある
kuruma["toiawase_datetime"] = pd.to_datetime(kuruma["b.toiawase_date"].str[:10] + " " + kuruma["b.toiawase_date"].str[10:])
kuruma["toiawase_y"] = kuruma["toiawase_datetime"].dt.year.astype(str)
kuruma["toiawase_m"] = kuruma["toiawase_datetime"].dt.month.astype(str)
kuruma["toiawase_ym"] = kuruma["toiawase_y"] + kuruma["toiawase_m"]

<b>
物件×問い合わせ元データ作成・・・ここまで<br>
物件×問い合わせ辞書作成・・・ここから<br>
（辞書じゃなくてサマった結果を民力と一緒に保持しておくのがいいかもしれない）
<b/>

In [51]:
#地域別集計(問い合わせ)
#全て辞書形式で持っておいて、実際の車に紐付ける形にする
#実際の元データの車種名、地域、スペックと紐付けたドア数をキーにして辞書から値を作成する
def convert_to_unit_based_num(data,colname,indice):
    out = data.reset_index().pivot(index=indice,columns=colname,values="count")
    vals = []
    #以上値の除去
    for i in out.columns:
        vals.append(out.columns.name+"_"+str(i))
    out.columns=vals
    out[indice] = out.index
    del out.index.name
    
    return out

In [45]:
#都道府県
pref_summary = kuruma.groupby("b.todofuken_jusho_kj").sum()["count"]
pref_summary_dicts = {key: value for (key, value) in zip(pref_summary.index.values,pref_summary.values)}
kuruma["num_toiawase_in_pref"] = kuruma["b.todofuken_jusho_kj"].apply(pref_summary_dicts.get)

#市区町村
city_summary = kuruma.groupby("city_name").sum()["count"]
city_summary_dicts = {key: value for (key, value) in zip(city_summary.index.values,city_summary.values)}
kuruma["num_toiawase_in_city"] = kuruma["city_name"].apply(city_summary_dicts.get)

#都道府県/市区町村単位でボディタイプ、問合年月、ドア数で集計（計６パターン）
shukei_val = ["a.body_type_name", "toiawase_ym", "a.door_su"]
out_name = ["body", "month", "door",]

pref_city = ["b.todofuken_jusho_kj","city_name"]
out_preposition = ["pref", "city"]

for s,o in zip(shukei_val,out_name):
    for p,op in zip(pref_city,out_preposition):
#             exec(op+'_'+o+'_summary = kuruma.groupby(["'+p+'","'+s+'"]).sum()["count"]') #groupby().sum()["count"]
#             exec(op+'_'+o+'_summary_dicts = {key: value for key, value in zip('+op+'_'+o+'_summary.index.values,'+op+'_'+o+'_summary.values)}') #辞書に格納
#             exec('kuruma["num_toiawase_'+o+'_in_'+p+'"] = kuruma.apply(lambda x :'+op+'_'+o+'_summary_dicts.get((x["'+p+'"],x["'+s+'"])), 1)') #変数を問い合わせに追加
#             exec(o+'_by_'+op+' = convert_to_unit_based_num('+op+'_'+o+'_summary'+',"'+str(s)+'","'+p+'")') #県別/市区町村別でテーブル作成
        su = str(op)+'_'+str(o)+'_summary' #中間df名
        sud = str(op)+'_'+str(o)+'_summary_dicts' #辞書名
        va = 'num_toiawase_'+str(o)+'_in_'+str(p) #変数名
        df2 = o+'_by_'+op #最終df名

        gl[su] = kuruma.groupby([p,s]).sum()["count"] #groupby().sum()["count"]
        gl[sud] = {key: value for key, value in zip(gl[su].index.values,gl[su].values)} #辞書に格納
        kuruma[va] = kuruma.apply(lambda x :gl[sud].get((x[p],x[s])), 1) #変数を問い合わせに追加
        gl[df2] = convert_to_unit_based_num(gl[su],s,p) #県別/市区町村別でテーブル作成

#出来上がったデータセットをすべてくっつける
cities = [body_by_city, door_by_city, month_by_city]
prefs = [body_by_pref, door_by_pref, month_by_pref]

#prefsの変数名を変更しておく必要がある
#遅いと思われるけど、シンプルさを優先した
for c in cities:
    koku_hoyu = pd.merge(koku_hoyu,c,on="city_name")
for p in prefs:
    p = p.rename(columns={"b.todofuken_jusho_kj":"chiho"})
    koku_hoyu = pd.merge(koku_hoyu,p,on="chiho")

koku_hoyu_area = koku_hoyu.copy()


In [47]:
body_by_city.head(2)

,a.body_type_name_その他,a.body_type_name_オープン,a.body_type_name_クロカン・ＳＵＶ,a.body_type_name_クーペ,a.body_type_name_ステーションワゴン,a.body_type_name_セダン,a.body_type_name_トラック,a.body_type_name_ハッチバック,a.body_type_name_ピックアップトラック,a.body_type_name_ミニバン,city_name
,3.0,9.0,24.0,31.0,7.0,31.0,5.0,61.0,2.0,28.0,
あきる野市,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,あきる野市


In [55]:
shashu_fmc_mc_summary = kuruma.groupby(["a.keisai_shashu_kj","a.keisai_fmc_cd","a.keisai_mc_cd"]).sum()["count"]
shashu_fmc_mc_summary_dicts = {key: value for (key, value) in zip(shashu_fmc_mc_summary.index.values,shashu_fmc_mc_summary.values)}
kuruma["num_shashu_fmc_mc"] = kuruma.apply(lambda x :shashu_fmc_mc_summary_dicts.get((x["a.keisai_shashu_kj"],x["a.keisai_fmc_cd"],x["a.keisai_mc_cd"])), 1)
#2種以上の変数を持つことになると問題が発生する
# fmc_mc_by_shashu = convert_to_unit_based_num(shashu_fmc_mc_summary,"a.keisai_shashu_kj",["a.keisai_fmc_cd","a.keisai_mc_cd"])


In [59]:
shashu_fmc_mc_summary.reset_index().head()
shashu_fmc_mc_summary[""]


,a.keisai_shashu_kj,a.keisai_fmc_cd,a.keisai_mc_cd,count
0,1007,F001,M001,1
1,1007,F001,M002,5
2,106,F001,M008,1
3,117クーペ,F999,M999,4
4,180SX,F001,M002,5


In [ ]:
#車種・グレード別集計（問い合わせ）
# -車種ごと
shashu_summary = kuruma.groupby("a.keisai_shashu_kj").sum()["count"]
shashu_summary_dicts = {key: value for (key, value) in zip(shashu_summary.index.values,shashu_summary.values)}
kuruma["num_shashu"] = kuruma["a.keisai_shashu_kj"].apply(shashu_summary_dicts.get)
# -車種&グレードごと
shashu_grade_summary = kuruma.groupby(["a.keisai_shashu_kj","a.keisai_grade_name"]).sum()["count"]
shashu_grade_summary_dicts = {key: value for (key, value) in zip(shashu_grade_summary.index.values,shashu_grade_summary.values)}
kuruma["num_shashu_grade"] = kuruma.apply(lambda x :shashu_grade_summary_dicts.get((x["a.keisai_shashu_kj"],x["a.keisai_grade_name"])), 1)
grade_by_shashu = convert_to_unit_based_num(shashu_grade_summary,"a.keisai_shashu_kj","a.keisai_grade_name")

#body_typeごと
body_summary = kuruma.groupby("a.body_type_name").sum()["count"]
body_summary_dicts = {key: value for (key, value) in zip(body_summary.index.values,body_summary.values)}
kuruma["num_body_type"] = kuruma["a.body_type_name"].apply(body_summary_dicts.get)
#おまけ：ドア数単位
door_summary = kuruma.groupby("a.door_su").sum()["count"]
door_summary_dicts = {key: value for (key, value) in zip(door_summary.index.values,door_summary.values)}
kuruma["num_door"] = kuruma["a.door_su"].apply(door_summary_dicts.get)
#車種・グレード別集計（お気に入り）

#地域みたいにまとめることができないので、個別のDFを吐き出しておく


In [39]:
# body_summary = kuruma.groupby("a.body_type_name").sum()["count"]
# body_summary_dicts = {key: value for (key, value) in zip(body_summary.index.values,body_summary.values)}
# kuruma["num_body_type"] = kuruma["a.body_type_name"].apply(body_summary_dicts.get)        
# # -車種ごと
# shashu_summary = kuruma.groupby("a.keisai_shashu_kj").sum()["count"]
# shashu_summary_dicts = {key: value for (key, value) in zip(shashu_summary.index.values,shashu_summary.values)}
# kuruma["num_shashu"] = kuruma["a.keisai_shashu_kj"].apply(shashu_summary_dicts.get)
# # -車種&FMCごと
# shashu_fmc_summary = kuruma.groupby(["a.keisai_shashu_kj","a.fmc_cd"]).sum()["count"]
# shashu_fmc_summary_dicts = {key: value for (key, value) in zip(shashu_fmc_summary.index.values,shashu_fmc_summary.values)}
# kuruma["num_shashu_fmc"] = kuruma.apply(lambda x :shashu_fmc_summary_dicts.get((x["a.keisai_shashu_kj"],x["a.fmc_cd"])), 1)
# fmc_by_shashu = convert_to_unit_based_num(shashu_fmc_summary,"a.keisai_shashu_kj","a.fmc_cd")
# # -車種&FMC&MCごと
# shashu_fmc_mc_summary = kuruma.groupby(["a.keisai_shashu_kj","a.fmc_cd","a.mc_cd"]).sum()["count"]
# shashu_fmc_mc_summary_dicts = {key: value for (key, value) in zip(shashu_fmc_mc_summary.index.values,shashu_fmc_mc_summary.values)}
# kuruma["num_shashu_fmc_mc"] = kuruma.apply(lambda x :shashu_fmc_mc_summary_dicts.get((x["a.keisai_shashu_kj"],x["a.fmc_cd"],x["a.mc_cd"])), 1)
# fmc_mc_by_shashu = convert_to_unit_based_num(shashu_grade_summary,"a.keisai_shashu_kj","a.fmc_cd","a.mc_cd")
# # -車種&FMC&MC&グレードごと
# shashu_fmc_mc_grade_summary = kuruma.groupby(["a.keisai_shashu_kj","a.fmc_cd","a.mc_cd","a.keisai_grade_name"]).sum()["count"]
# shashu_fmc_mc_grade_summary_dicts = {key: value for (key, value) in zip(shashu_fmc_mc_grade_summary.index.values,shashu_fmc_mc_grade_summary.values)}
# kuruma["num_shashu_fmc_mc_grade"] = kuruma.apply(lambda x :shashu_fmc_mc_grade_summary_dicts.get((x["a.keisai_shashu_kj"],x["a.fmc_cd"],x["a.mc_cd"],x["a.keisai_grade_name"])), 1)
# grade_fmc_mc_by_shashu = convert_to_unit_based_num(shashu_fmc_mc_grade_summary,"a.keisai_shashu_kj","a.fmc_cd","a.mc_cd","a.keisai_grade_name")

In [37]:
kuruma.head()

,a.keisai_shashu_kj,a.keisai_grade_name,a.door_su,a.body_type_name,a.mission_flg,a.mt_mode_at_flg,a.handle_flg,a.air_conditioner_flg,a.power_steering_flg,a.power_window_flg,a.air_bag_flg,a.joshuseki_air_bag_flg,a.side_air_bag_flg,a.abs_flg,a.esc_flg,a.keyless_flg,a.md_flg,a.cd_flg,a.cd_navigation_flg,a.dvd_navigation_flg,a.hdd_navigation_flg,a.katashiki,b.toiawase_date,b.todofuken_cd,b.todofuken_jusho_kj,b.shikuchoson_jusho_kj,b.zip,b.hojin_cd,b.madoguchi_cd,b.yyyymmdd,city_name,count,toiawase_datetime,toiawase_y,toiawase_m,toiawase_ym,num_toiawase_in_pref,num_toiawase_in_city,num_toiawase_body_in_b.todofuken_jusho_kj,num_toiawase_body_in_city_name,num_toiawase_month_in_b.todofuken_jusho_kj,num_toiawase_month_in_city_name,num_toiawase_door_in_b.todofuken_jusho_kj,num_toiawase_door_in_city_name,num_shashu,num_shashu_grade,num_body_type,num_door
0,F355,GTS,2.0,クーペ,10,0,2,1,1,1,0,0,0,0,0,0,0,1,0,0,0,9999,2015-06-2408:35:20,36,神奈川県,横浜市神奈川区,null,220294,1,20161213,横浜市神奈川区,1,2015-06-24 08:35:20,2015,6,20156,305,12,58,3,6,1,61,5,5,1,649,858
1,ピアッツァ,1.8XE/Sハンドリングバイロータス,3.0,クーペ,2,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,E-JT221F,2016-04-2621:33:34,35,東京都,港区麻布十番,1060045,31078,1,20161213,港区,1,2016-04-26 21:33:34,2016,4,20164,430,28,74,8,10,5,37,1,2,1,649,517
2,シルビア,2.0Q’sSE,2.0,クーペ,9,0,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,E-S14,2015-03-0313:38:33,53,大阪府,東大阪市,null,210545,1,20161213,東大阪市,1,2015-03-03 13:38:33,2015,3,20153,318,17,32,2,28,2,58,3,42,1,649,858
3,ミニ,MK-Ⅰ仕様,3.0,ハッチバック,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,E-XN12A,2015-06-0215:07:26,45,長野県,安曇野市穂高,3998303,214352,4,20161213,安曇野市,1,2015-06-02 15:07:26,2015,6,20156,123,5,47,3,7,2,20,1,58,1,1667,517
4,5シリーズ,525iハイライン,4.0,セダン,3,0,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,GH-DT25,2015-08-1009:00:28,36,神奈川県,横浜市神奈川区二ツ谷町,2210823,40648,1,20161213,横浜市神奈川区,1,2015-08-10 09:00:28,2015,8,20158,305,12,47,2,9,3,59,2,26,1,706,889


In [40]:
# shashu_summary.reset_index().ix[shashu_summary.reset_index()["a.keisai_shashu_kj"]=="ムーヴ"]
shashu_grade_summary.reset_index().head()
ku = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/物件*問い合わせ/toiawase_1.csv",
                         delimiter="\t", dtype="object", encoding="cp932",nrows=10000)
ku.head()

,a.keisai_shashu_kj,a.keisai_shashu_brand_cd,a.keisai_grade_name,a.keisai_fmc_cd,a.keisai_mc_cd,a.keisai_shashu_ren_no,a.keisai_haikiryo_grade_cd,a.door_su,a.body_type_name,a.body_type_cd,a.mission_flg,a.mt_mode_at_flg,a.handle_flg,a.air_conditioner_flg,a.power_steering_flg,a.power_window_flg,a.air_bag_flg,a.joshuseki_air_bag_flg,a.side_air_bag_flg,a.abs_flg,a.esc_flg,a.keyless_flg,a.md_flg,a.cd_flg,a.cd_navigation_flg,a.dvd_navigation_flg,a.hdd_navigation_flg,a.katashiki,b.toiawase_date,b.todofuken_cd,b.todofuken_jusho_kj,b.shikuchoson_jusho_kj,b.zip,b.hojin_cd,b.madoguchi_cd,b.yyyymmdd
0,F355,FE,GTS,F001,M001,S006,G002,2.0,クーペ,C,10,0,2,1,1,1,0,0,0,0,0,0,0,1,0,0,0,9999,2015-06-24 08:35:20,36,神奈川県,横浜市神奈川区,null,220294,001,20170103
1,ピアッツァ,IS,1.8 XE/S ハンドリング バイ ロータス,F999,M999,S009,G999,3.0,クーペ,C,02,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,E-JT221F,2016-04-26 21:33:34,35,東京都,港区麻布十番,1060045,031078,001,20170103
2,シルビア,NI,2.0 Q’s SE,F002,M003,S051,G025,2.0,クーペ,C,09,0,1,1,1,1,1,0,0,0,0,0,0,1,0,0,0,E-S14,2015-03-03 13:38:33,53,大阪府,東大阪市,null,210545,001,20170103
3,ミニ,RV,MK-Ⅰ仕様,F001,M002,S001,G999,3.0,ハッチバック,D,02,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,E-XN12A,2015-06-02 15:07:26,45,長野県,安曇野市穂高,3998303,214352,004,20170103
4,5シリーズ,BM,525i ハイライン,F002,M005,S017,G003,4.0,セダン,S,03,0,1,1,1,1,1,1,1,1,0,1,0,1,0,1,0,GH-DT25,2015-08-10 09:00:28,36,神奈川県,横浜市神奈川区二ツ谷町,2210823,040648,001,20170103


<b>------------------------------問い合わせデータ＆辞書作成ーここまで------------------------------<b/><br>
<b>------------------------------成約データ読み込み(お気に入りの車種別集計のために必要)ーここから---------------------------<b/>

In [17]:
moto = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/加工済み/raw_data.csv",delimiter=" ")
print(moto.shape)
#きちんとテストする際に全量で実施する必要あり
#要修正
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 123)
moto.head()
#shashuとchihoに上段で作成したデータを紐付ける

# 地域データ：koku_hoyu_area_cusdb（マージキー：chiho）
# 車データ：shashu_summary, grade_by_shashu, body_summary, door_summary(マージキー：車種、車種、ボディタイプ、ドア数)
# for i in moto.columns:
#     print(i)


(10000, 123)


,wns_bukken_cd,shinkoyo_nohimbi_cd,hojin_cd,madoguchi_cd,chiho,coupon_flg,fair_flg,sd_flg,kuchikomi_flg,maker_name,shashu,brand_cd,renno,fmc,mc_cd,grade_cd,grade_kukuri_cd,body_type_cd,body_type_name,kei_jidosha_flg,haikiryo,year,month,distance,price,total_price,csa_hosho_flg,str_hosho_flg,plan_flg,grade_hoki_flg,movie_flg,mylist_msg_flg,keisai_comment_flg,nintei_flg,gazo_cnt,caption_cnt,y_nintei_flg,kakudai_flg,keisai_shufukureki_hyojikbn_cd,ksai_shaken_tukinashi_cd,shaken_hyojiyo,shaken_kensakuyo,keisai_iro_kj,white_flg,pearl_flg,red_flg,blue_flg,dark_blue_flg,green_flg,black_flg,brown_flg,gold_flg,silver_flg,yellow_flg,orange_flg,peach_flg,purple_flg,gray_flg,sonota_flg,si_iro_kj,mission_flg,mt_mode_at_flg,handle_flg,air_conditioner_flg,power_steering_flg,power_window_flg,air_bag_flg,joshuseki_air_bag_flg,side_air_bag_flg,abs_flg,esc_flg,keyless_flg,md_flg,cd_flg,cd_navigation_flg,dvd_navigation_flg,hdd_navigation_flg,sunroof_flg,hongawa_sheet_flg,etc_flg,back_monitor_flg,dis_charged_lamp_flg,full_aero_flg,low_down_flg,aluminum_wheel_flg,nonsmoking_car_flg,one_owner_flg,dealer_car_flg,teiki_tenkenbo_flg,fukushisha_flg,four_wheel_drive_flg,seibi_flg,hosho_flg,memory_navigation_flg,n_parallel_importcar_flg,u_parallel_importcar_flg,endgine_flg,turbo_flg,w_aircon_flg,tv_flg,kanreichi_flg,tounan_boushi_flg,dendo_seat_flg,fullflat_seat_flg,bench_seat_flg,sanretsu_seat_flg,walk_threw_flg,mishiyosha_flg,slide_door_flg,same_shashu,keisai_days,del_cd,del_date,y_cs,p_cs,y_other,p_other,kuchikomi_cnt,katashiki,cnt_total_price,avg_total_price,min_total_price,predicted_price
810772,U00012656926,150102,303968,1,福岡県,1,0,0,1,ダイハツ,ムーヴ,DA,S012,F003,M001,G010,K054,D,ハッチバック,1.0,660,2003,2,107000,289000.0,389000.0,0,1,0,0,0,0,1,0,15,9,0,4,1,1.0,車検整備付,0,グレーメタリック,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,灰Ｍ,13,0,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,T,0,0,0,0,0,1,1,0,0,0,0,1.018684,100,C99,20150718.0,0,0.037452,0,0.148028,4,LA-L152S,445,3.382966e+05,80000.0,3.140288e+05
5301564,UZ0014935617,150403,219007,1,山梨県,0,0,0,1,ホンダ,レジェンド,HO,S033,F003,M005,G003,K019,S,セダン,0.0,3700,2011,11,29000,2980000.0,3212000.0,0,1,0,1,0,0,0,0,19,7,0,1,0,NaN,28.11,19,アルバータホワイトパール,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,白真珠,4,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,1,0,1,1,1,1,0,0,1,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1.056126,126,C97,20150806.0,0,0.074008,0,0.120010,4,DBA-KB2,1,3.212000e+06,3212000.0,2.410772e+06
4924075,UZ0013173022,141107,51721,115,宮城県,1,1,0,1,スバル,R1,SB,S001,F001,M005,G010,K016,D,ハッチバック,1.0,660,2010,3,5000,1459000.0,1558000.0,0,1,0,0,0,0,1,0,8,8,0,1,0,NaN,27.3,4,シルキーホワイトパール,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,真珠白,33,0,1,1,1,1,1,1,0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,2,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.031670,126,C02,20150313.0,0,0.026905,0,0.124821,16,DBA-RJ2,156,1.082494e+06,328000.0,1.041860e+06
4500543,UD0015293572,150703,81727,5,滋賀県,0,0,0,0,スズキ,ソリオ,SZ,S014,F002,M002,G001,K007,D,ハッチバック,0.0,1200,2013,3,52000,780000.0,900000.0,0,1,1,0,0,0,1,0,18,0,0,1,0,NaN,28.3,8,シルバー,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,銀,33,0,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.009649,26,E11,20150903.0,0,0.011554,0,0.144302,0,DBA-MA15S,72,1.242056e+06,739000.0,9.695125e+05
4635857,UJ0013369648,141003,30121,1,愛知県,1,1,0,1,三菱,ミニキャブバン,MI,S016,F003,M009,G005,K010,D,ハッチバック,1.0,660,2007,6,61000,398000.0,534000.0,0,1,1,1,0,1,1,0,17,12,0,4,0,1.0,車検整備付,0,ホワイトソリッド,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,白,1,0,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.010482,12,C01,20141014.0,0,0.032983,0,0.286805,54,GBD-U61V,1001,7.178082e+05,150000.0,4.494569e+05


<b>------------------------------成約データ読み込みーここまで------------------------------<b/><br>
<b>------------------------------お気に入りデータ作成ー開始---------------------------<b/>

In [47]:
#accessDBの形なので、取り込みを頑張る必要あり
#→無理なのでWinでcsvにしてから取り込むことに
#dask使わないと遅すぎて無理
#元データ作成
col = ["wns_bukken_cd", "shinkoyo_nohimbi_cd", "han_cd", "hojin_cd", "madoguchi_cd", "sum_toiawase",
       "base_plan_toiawase", "plan1_toiawase", "plan2_toiawase", "cart_add", "cart_delete"]

start = time.time()
tes1 = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/from花香さん/csv/001.csv",names=col)
tes2 = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/from花香さん/csv/002.csv",names=col)
tes3 = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/from花香さん/csv/003.csv",names=col)
tes4 = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/from花香さん/csv/004.csv",names=col)
tes5 = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/from花香さん/csv/005.csv",names=col)

t1 = tes1.append(tes2)
t2 = t1.append(tes3)
t3 = t2.append(tes4)
t4 = t3.append(tes5)

t5 = t4.loc[:,["wns_bukken_cd", "shinkoyo_nohimbi_cd", "sum_toiawase", "cart_add", "cart_delete"]]
# t5.head(10)

In [49]:
# データが多すぎて先に車種単位にまとめないと処理不可能
#物件tblからデータを抜いてきて（wns_bukken_cd,車種名、車種に関連するものだけ）突合、groupbyで車種ごとにサマるところからスタート
tobe = moto.ix[:,["wns_bukken_cd", "shashu", "grade_cd", "grade_kukuri_cd", "body_type_cd", "body_type_name"]]
#rightはpandas DataFrameじゃないと何故か上手くいかない・・・
bukken = t5.merge(tobe,how="inner",on="wns_bukken_cd")
bukken["count"]=1

In [50]:
#全体のお気に入り数を出す
zentai = bukken.groupby(["shashu","grade_cd"]).sum()
zen = zentai.reset_index()
zen["shinkoyo_nohimbi_cd"] = "all"
zen["cart_sum"] = zen["cart_add"] - zen["cart_delete"]
zen.head()

/Users/01017387/anaconda/lib/python3.5/site-packages/pandas/core/groupby.py:4022: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


,shashu,grade_cd,shinkoyo_nohimbi_cd,sum_toiawase,cart_add,cart_delete,count,cart_sum
0,1007,G002,all,4,7,1,48,6
1,1007,G003,all,2,10,2,8,8
2,1007,G999,all,0,2,1,25,1
3,106,G001,all,2,11,6,43,5
4,117クーペ,G999,all,3,45,15,28,30


In [56]:
#期間を絞って（１年分、１ヶ月単位）お気に入り数の遷移を見る
#最終的には車種、グレード、nohimbiをキーとした辞書を作成し、車種ごとのお気に入り数を当てる感じになる
#死ぬほど時間かかるので、今回は飛ばしでいいかも

# shashuとgrade_cdでgroupbyする
sam = bukken.groupby(["shashu","grade_cd","shinkoyo_nohimbi_cd"]).sum()
#元テーブルとして使う
samr = sam.reset_index()
#shinkoyo_nohinbiが特定日付区間に入っているものだけを抜き出して、groupby(shashu,grade_cd)で処理する
upper_date = 151030
#upper_dateが100ずつ足されていく感じ
#dataframeに入ってる最終日を超えたら処理中断
# limit = int(t5["shinkoyo_nohimbi_cd"][len(t5)-1])
limit = int(t5["shinkoyo_nohimbi_cd"].tail().iloc[4])
loop_count=0

while upper_date <= limit:
    lower = samr[samr["shinkoyo_nohimbi_cd"] > (upper_date - 10000)]
    lowup = lower[lower["shinkoyo_nohimbi_cd"] <= upper_date]

    #日付を絞ってループで回す
    if len(lowup) >0:
        smpl = lowup
    else:
        pass
    
    #実際の集計
    part = smpl.groupby(["shashu","grade_cd"]).sum()
    datum = part.reset_index()
    #変数名の加工

    dt = str(smpl["shinkoyo_nohimbi_cd"].tail().iloc[4])
    col_list = ["sum_toiawase", "cart_add", "cart_delete", "count"]
    new_col = ["shashu", "grade_cd", "shinkoyo_nohimbi_cd"]
    for k in col_list:
        new_col.append("from_"+str(dt)+"_1y_back_"+k)

    datum.columns = new_col
    datum = datum.drop("shinkoyo_nohimbi_cd",axis=1)
    datum = datum.drop("count",axis=1)
    datum["favorite"] = datum["cart_add"] - datum["cart_delete"]
    upper_date +=100
    if int(upper_date/100%100) ==13:
        upper_date = upper_date + 8700 #年を更新する処理

    #初回ループでなければデータをマージしていく
    #要修正
    #マージでなくて、shinkoyo_nohimbi_cdの違うものが縦に積みあがるケースも考慮すべき
    
    if loop_count != 0:
#         okini.merge(datum,on=["shashu","grade_cd"])
        okini.merge(datum,how="outer",on=["shashu","grade_cd"],)
    else:
        okini = datum
    loop_count +=1
    
#     if loop_count >2:
#         break
    
#あとはこれをshashu,grade_cdで横に繋げていく(merge)
#一番重い処理なので、回さないほうがいいかも

/Users/01017387/anaconda/lib/python3.5/site-packages/pandas/core/groupby.py:4022: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


In [59]:
# print(datum.shape)
datum.head()

/Users/01017387/anaconda/lib/python3.5/site-packages/pandas/core/groupby.py:4022: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


,shashu,grade_cd,shinkoyo_nohimbi_cd,from_6473966_1y_back_sum_toiawase,from_6473966_1y_back_cart_add,from_6473966_1y_back_cart_delete,from_6473966_1y_back_count
0,1007,G002,5869916,4,7,1,39
1,1007,G999,3759301,0,2,1,25
2,106,G001,5275644,2,7,4,35
3,117クーペ,G999,3462126,3,45,15,28
4,180SX,G003,3909403,3,17,6,26


<b>------------------------------お気に入りデータ作成ーここまで------------------------------<b/><br>
<b>------------------------------カタログ作成ー開始---------------------------<b/>

In [23]:
#全部で21種
cat_list = ["mc1_body_type.txt", "mc1_kanren_shashu.txt","mc1_brand.txt", "mc1_kihon_iro.txt","mc1_brand_country.txt",
            "mc1_kukuri_kanren_shashu.txt","mc1_brand_iro.txt", "mc1_maker.txt","mc1_brand_maker.txt", "mc1_mc_model.txt",
            "mc1_country.txt", "mc1_mc_model_gazo.txt","mc1_fmc_model.txt", "mc1_net_category.txt",
            "mc1_fp_shashu_midashi_settei_work.txt", "mc1_shashu.txt","mc1_grade.txt", "mc1_sobi.txt","mc1_grade_iro.txt",
            "mc1_transmission_henkan.txt","mc1_grade_sobi.txt"]
index_name =["ボディ形状マスタ", "関連車種マスタ", "ブランドマスタ", "基本色マスタ", "ブランド国設定マスタ", "括り関連車種マスタ",
             "ブランド色マスタ", "メーカーマスタ", "ブランドメーカー設定マスタ", "MCモデルマスタ", "国マスタ", "MCモデル画像マスタ",
             "FMCモデルマスタ", "NETカテゴリマスタ", "FP車種見出し設定マスタ", "車種マスタ", "グレードマスタ", "装備マスタ",
             "グレード色設定マスタ", "トランスミッション変換マスタ", "グレード装備設定マスタ", ]
df_columns = [["body_shape_cd", "body_shape_nm", "register_ymd", "last_modified_ymd"],
              ["kanren_shashu_cate", "kanren_shashu_cate_no", "kanren_kj", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "brand_nm", "brand_hyoji_order", "catalogue_keisai_fuka_cate", "seo_folder_nm", "register_ymd", "last_modified_ymd"], 
              ["base_color_cd", "base_color_nm", "base_color_hyoji_jun", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "country_cd", "daihyo_country_flg", "register_ymd", "last_modified_ymd"], 
              ["kanren_shashu_cate", "kanren_shashu_cate_no", "brand_cd", "shashu_cd", "kukuri_kanren_shashu_hyoji_jun", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "brand_color_cd", "brand_color_nm", "base_color_cd", "jitsu_brand_color_nm", "brand_color_hyoji_order", "base_color_cd1", "base_color_cd2", "register_ymd", "last_modified_ymd"], 
              ["maker_cd", "maker_nm", "maker_hyoji_jun", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "maker_cd", "daihyo_maker_flg", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "mc_hanbai_start", "mc_hanbai_end", "mc_cate", "mc_kj", "mc_commment_title", "mc_comment", "catalogue_keisai_fuka_cate", "register_ymd", "last_modified_ymd"], 
              ["country_cd", "country_nm", "area_cate", "country_hyoji_jun", "register_ymd", "daihyo_base_color", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "gazo_index_no", "mc_model_pic_hyoji_jun", "gazo_cate_cd", "caption", "catalogue_keisai_fuka_cate", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "fmc_hanbai_start", "fmc_hanbai_end", "fmc_kj", "catalogue_keisai_fuka_cate", "register_ymd", "last_modified_ymd"], 
              ["net_cate_cd", "cate_nm", "net_cate_hyoji_jun", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "fp_cate_cd", "fp_shashu_midashi_cd", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "shashu_nm", "wareki_seireki_cate", "shashu_hyoji_jun", "catalogue_keisai_fuka_cate", "search_key_1", "search_key_2", "search_key_3", "seo_folder_nm", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "grade_kukuri_cd", "grade_nm", "grade_hyoji_jun", "body_shape_cd", "grade_hanbai_start_ym", "grade_hanbai_end_ym", "katashiki", "ruibetsu_kigo", "katashiki_shitei_no", "ruibetsu_cate_no", "price_with_tax", "price_without_tax", "nenpi_kijun_achieve_cate", "nenpi_kijun_nen", "low_emission_nintei_stars", "low_emission_nintei_nen", "reduce_tax_cate", "heiko_yunyusha_cate", "walfare_car_cate", "car_for_commercial_use", "kei_flg", "number_size", "special_edition_flg", "catalogue_keisai_fuka_cate", "keisai_commentary", "kudo_hoshiki_cate", "transmission_su", "transmission_cate", "transmission_ichi_cate", "transmission_cd", "handle_ichi_cate", "length", "width", "height", "inner_length", "inner_width", "inner_height", "wheel_base", "front_tred", "back_tred", "min_shako", "weight_of_car", "weight_of_car_all", "max_carriage", "max_carriage_persons", "min_kaiten_hankei", "10_15_mode_nenpiritsu", "stearing_gear_cate", "stearing_gear_nm", "front_suspension_cate", "front_suspension_nm", "back_suspension_cate", "back_suspension_nm", "front_break_cate", "front_break_nm", "back_break_cate", "back_break_nm", "front_tire_txt", "back_tire_txt", "engine_katashiki", "engine_valve_cate", "engine_silinder_cate", "engine_kito_su", "engine_kakyuki_cate", "engine_cate", "silinder_radius", "silinder_koutei", "total_haikiryo", "haikiryo1", "haikiryo2", "asshukuhi", "max_shuturyoku_kw", "max_shuturyoku_ps", "max_shuturyoku_rmp_max", "max_shuturyoku_rmp_min", "max_torque_mn", "max_torque_kgm", "max_torque_rpm_max", "max_torque_rpm_min", "fuel_sapply_device_cate", "fuel_cate", "fuel_tank_capacity", "door_su", "seat_columns", "net_cate_cd", "register_ymd", "last_modified_ymd", "jc08_mode_nenpi_rtsu"], 
              ["equipment_cd", "equipment_nm", "equipment_bunrui_cate", "catalogue_keisai_cate", "equipment_hyoji_jun", "equipment_usage_cd", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "brand_color_cd", "option_flg", "option_price", "grade_hyoji_jun", "register_ymd", "last_modified_ymd"], 
              ["transmission_cd", "transmission_su", "transmission_cate", "transmission_ichi_cate", "register_ymd", "last_modified_ymd"], 
              ["brand_cd", "shashu_cd", "fmc_cd", "mc_cd", "grade_cd", "equipment_cd", "equipment_nm", "equipment_jokyo_cd", "nm", "register_ymd", "last_modified_ymd"]
             ]
#読んだ後にとりあえず全部くっつけたい
co = 0
df_names = []
for ct in cat_list:
    df_names.append(ct.rstrip(".txt")[4:])

    
start = time.time
dfs = []
cnt = 0
for cat,dn,dc in zip(cat_list,df_names,df_columns):
    var = 'cat_'+str(dn)#変数名
    tgt = '/Users/01017387/Desktop/しごと/data/car用/車カタログ/'+str(cat)

    if cat in ["mc1_grade_sobi.txt", "mc1_grade_iro.txt", "mc1_brand_iro.txt"]:
        gl[var] = pd.read_csv(tgt,encoding="cp932",delimiter="\t",names=dc)
#         exec('cat_'+str(dn)+' = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/'+str(cat)+'",encoding="cp932",delimiter="\t",names='+str(dc)+')')
    #型指定がうまくいかなくてエラーになるので、すべてobjectで読み込む
    else:
        gl[var] = dskdf.read_csv(tgt,encoding="cp932",delimiter="\t",dtype="object",names=dc)
#         exec('cat_'+str(dn)+' = dskdf.read_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/'+str(cat)+'",encoding="cp932",delimiter="\t",low_memory=False,dtype="object",names='+str(dc)+')')
    cnt +=1
    #登録日、更新日は不要なので読み込み時点で落とす
#     exec('"cat_"+'+str(dn)+'["last_modified_tmd"]')
    
    #データ名もリストに格納しておく
    dfs.append("cat_"+str(dn))
#     print('cat_'+str(dn))
#     print(cat)
#     co +=1
#     if co ==17:
#         break

#少しでも軽くするために、不要な変数を削除しておく
for dd in dfs:
    print(dd)
    gl[dd] = gl[dd].drop("register_ymd",axis=1)
    gl[dd] = gl[dd].drop("last_modified_ymd",axis=1)
    if dd == "cat_grade":
        gl[dd] = gl[dd].drop("number_size",axis=1)
        gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate",axis=1)
    if dd == "cat_mc_model":
        gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate",axis=1)
    if dd == "cat_shashu":
        gl[dd] = gl[dd].drop("catalogue_keisai_fuka_cate",axis=1)
        gl[dd] = gl[dd].drop("search_key_1",axis=1)
        gl[dd] = gl[dd].drop("search_key_2",axis=1)
        gl[dd] = gl[dd].drop("search_key_3",axis=1)
        gl[dd] = gl[dd].drop("seo_folder_nm",axis=1)
    if dd == "cat_sobi":
        gl[dd] = gl[dd].drop("equipment_hyoji_jun",axis=1)


# for dd in dfs:
#     exec(dd+'.drop("register_ymd",axis=1)')
#     exec(dd+'.drop("last_modified_ymd",axis=1)')
#     if dd == "cat_grade":
#         exec(dd+'.drop("number_size",axis=1)')
#         exec(dd+'.drop("catalogue_keisai_fuka_cate",axis=1)')
#     if dd == "cat_mc_model":
#         exec(dd+'.drop("catalogue_keisai_fuka_cate",axis=1)')
#     if dd == "cat_shashu":
#         exec(dd+'.drop("catalogue_keisai_fuka_cate",axis=1)')
#         exec(dd+'.drop("search_key_1",axis=1)')
#         exec(dd+'.drop("search_key_2",axis=1)')
#         exec(dd+'.drop("search_key_3",axis=1)')
#         exec(dd+'.drop("seo_folder_nm",axis=1)')
#     if dd == "cat_sobi":
#         exec(dd+'.drop("equipment_hyoji_jun",axis=1)')

/Users/01017387/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


cat_body_type
cat_kanren_shashu
cat_brand
cat_kihon_iro
cat_brand_country
cat_kukuri_kanren_shashu
cat_brand_iro
cat_maker
cat_brand_maker
cat_mc_model
cat_country
cat_mc_model_gazo
cat_fmc_model
cat_net_category
cat_fp_shashu_midashi_settei_work
cat_shashu
cat_grade
cat_sobi
cat_grade_iro
cat_transmission_henkan
cat_grade_sobi


In [ ]:
#紐付け方を考える必要がある
#最終的には車種ごとに出したいので、車種から作っていく

#主に車種コードで紐付ける-shashu_cd
cat_grade_sobi(グレード装備マスタ)
→["pandasで作成、pivotして装備ごとのカラムを作ってから実施"]
cat_grade_iro(グレード色マスタ)(ブランドコード、車種コード、FMCコード、MCコード、グレードコード)
→["pandasで作成、cat_brand_iroとブランド色コードでマージしてから、columnsにbase_color_cd、valuesにcountを置いたものをマージする"]
→上記２つをマージ、daskに中身を入れて処理続行

cat_grade(グレードマスタ)(ブランドコード、車種コード、FMCコード、MCコード、グレードコード)
cat_mc_model(MCモデルマスタ)(ブランドコード、車種コード、FMCコード、MCコード)
cat_fmc_model(FMCモデルマスタ)(ブランドコード、車種コード、FMCコード)
cat_shashu(車種マスタ)(ブランドコード、車種コード)
cat_kukuri_kanren_shashu(括り関連車種マスタ)・・・類似車種のデータが入っている(ブランドコード、車種コード)
↑
# ボディ形状コードで紐付ける-body_shape_cd
cat_body_type
↑
#トランスミッションコードで紐付ける-transmission_cd
cat_transmission_henkan
↑
# ブランドコードでの紐付け-brand_cd
cat_brand_country(ブランド国設定マスタ)(ブランドコード)
cat_brand_maker(ブランドメーカーマスタ)(ブランドコード)
cat_brand(ブランドマスタ)(ブランドコード)
↑
# 関連車種区分で紐付ける-kanren_shashu_cate
cat_kanren_shashu


In [24]:
#装備ごとに作られているテーブルが存在するので、装備をダミー変数化するためにpivotする必要がある
#ダミー化した変数の内容がわかるように、全てにsobi_の接頭辞をつけておく
cat_grade_sobi_pv = pd.pivot_table(cat_grade_sobi,index=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"],columns = "equipment_nm",values = "equipment_jokyo_cd").reset_index()
col_equip = ["sobi_"+str(x) for x in cat_grade_sobi_pv.columns[5:,]]
bs = ['brand_cd','shashu_cd','fmc_cd','mc_cd','grade_cd']
bs.extend(col_equip)
cat_grade_sobi_pv.columns = bs

#grade_iroは先にbrand_iroとマージして、基本色を獲得しておく
cat_grade_iro["count"] = 1
cat_grade_brand_iro =pd.merge(cat_grade_iro,cat_brand_iro.ix[:,["brand_cd","brand_color_cd","base_color_cd"]],on=["brand_cd","brand_color_cd"])
sum_grade_brand_iro = pd.pivot_table(cat_grade_brand_iro,index=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"],columns="base_color_cd",values="count").reset_index()

#ダミー化した変数の内容がわかるように、全てにbase_color_の接頭辞をつけておく
col_colors = ["base_color_"+str(x) for x in sum_grade_brand_iro.columns[5:,]]
bs = ['brand_cd','shashu_cd','fmc_cd','mc_cd','grade_cd']
bs.extend(col_colors)
sum_grade_brand_iro.columns = bs
sum_grade_brand_iro.head()

,brand_cd,shashu_cd,fmc_cd,mc_cd,grade_cd,base_color_BG,base_color_BK,base_color_BL,base_color_BR,base_color_GL,base_color_GN,base_color_GY,base_color_NV,base_color_OR,base_color_PK,base_color_PP,base_color_RD,base_color_SL,base_color_WT,base_color_YL
0,ME,S014,F004,M005,G002,NaN,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN
1,ME,S014,F004,M005,G003,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
2,ME,S014,F004,M005,G004,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
3,ME,S014,F004,M005,G005,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN
4,ME,S014,F004,M005,G006,NaN,1.0,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN


In [26]:
#上記前処理が終わったら、ループで各種データをマージする

merge_key = [["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"],["brand_cd","shashu_cd","fmc_cd","mc_cd"],
             ["brand_cd","shashu_cd","fmc_cd"],["brand_cd","shashu_cd"],["brand_cd","shashu_cd"],
             ["body_shape_cd"],["transmission_cd"],["brand_cd"],["brand_cd"],["brand_cd"],["kanren_shashu_cate"]
            ]

merge_data = ["cat_grade","cat_mc_model","cat_fmc_model","cat_shashu","cat_kukuri_kanren_shashu","cat_body_type",
              "cat_transmission_henkan","cat_brand_country","cat_brand_maker","cat_brand","cat_kanren_shashu"
             ]

loops = np.array(list(range(11)))+2

pandas_list=["mc1_body_type.txt", "mc1_kanren_shashu.txt"]

#dask_ver
# a1 = cat_grade_sobi.merge(cat_grade_iro,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
#pandas_ver
moto = pd.merge(cat_grade_sobi_pv,sum_grade_brand_iro,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
#a1をdask化
a1 = dskdf.from_pandas(moto,npartitions=1)

cnt = 0
#dask_ver
for mk,md,lp in zip(merge_key,merge_data,loops):
#     a2 = a1.merge(cat_grade,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
    var = 'a'+str(lp)
    var_1 = 'a'+str(lp-1)
    gl[var] = gl[var_1].merge(gl[md],on=mk)
#     exec('a'+str(lp)+' = a'+str(lp-1)+'.merge('+md+',on='+str(mk)+')')
    
#pandas_ver
# for mk,md,lp in zip(merge_key,merge_data,loops):
# #     a2 = a1.merge(cat_grade,on=["brand_cd","shashu_cd","fmc_cd","mc_cd","grade_cd"])
#     exec('a'+str(lp)+' = pd.merge(a'+str(lp-1)+','+md+',on='+str(mk)+')')


#     cnt +=1
#     if cnt >=3:
#         break


In [27]:
a12.head()

,brand_cd,shashu_cd,fmc_cd,mc_cd,grade_cd,sobi_1000Wバング＆オルフセン ベオサウンドオーディオシステム,sobi_10インチボストンアコースティックス社製プレミアムオーディオシステム,sobi_2列目ウォークスルー,sobi_2列目シート,sobi_3列目シート,...,country_cd,daihyo_country_flg,maker_cd,daihyo_maker_flg,brand_nm,brand_hyoji_order,catalogue_keisai_fuka_cate_y,seo_folder_nm,kanren_shashu_cate_no_y,kanren_kj
0,ME,S014,F004,M005,G002,NaN,NaN,12.0,22.0,20.0,...,GER,1,013,1,メルセデス・ベンツ,19,0,mercedes-benz,59,ライバル-マツダロードスター・トヨタMR-S・ホンダS2000・BMWZ4・M・ベンツSLK
1,ME,S014,F004,M005,G002,NaN,NaN,12.0,22.0,20.0,...,GER,1,013,1,メルセデス・ベンツ,19,0,mercedes-benz,60,ライバル-スバルR1・スマートフォーツー
2,ME,S014,F004,M005,G002,NaN,NaN,12.0,22.0,20.0,...,GER,1,013,1,メルセデス・ベンツ,19,0,mercedes-benz,61,ライバル-スバルインプレッサ・三菱ランサーエボリューション
3,ME,S014,F004,M005,G002,NaN,NaN,12.0,22.0,20.0,...,GER,1,013,1,メルセデス・ベンツ,19,0,mercedes-benz,62,ライバル-スバルアウトバック・ボルボXC70・アウディオールロードクワトロ
4,ME,S014,F004,M005,G002,NaN,NaN,12.0,22.0,20.0,...,GER,1,013,1,メルセデス・ベンツ,19,0,mercedes-benz,63,ライバル-三菱コルトプラス・カローラフィールダー・ウィングロード・エアウェイブ


In [ ]:
#ファイルが死ぬほど大きいので、pickleで退避させておく
# with open('car_catalogue_dd.pkl','wb') as f1:
#     pickle.dump(a12,f1)

# with open('car_catalogue_dd.pkl','rb') as f2:
#     data=pickle.load(f2)
a12.to_csv("/Users/01017387/Desktop/しごと/data/car用/車カタログ/test/car_catalogue_ss.csv")
# a17.to_hdf("/Users/01017387/Desktop/しごと/data/car用/車カタログ/test/car_catalogue_ss.hdf")


<b>------------------------------カタログ作成ーここまで------------------------------<b/><br>
<b>------------------------------カスタマDB作成ー開始---------------------------<b/>

In [17]:
#フラグについては、とりあえず８割以上の確率が入っているものor真の値が入っているものを使う
#幼児フラグはあるものだけ使う
#フラグ対象：小学生以下子供フラグ、社会人フラグ、性別、カップル、
# have_child_integration_flg, sex_integration, age_integration_cd, high_income_predict_flg,
# businessperson_integration_flg, couple_flg, have_baby_flg, 
# car系フラグ #suumoはほとんど入ってないからもはや入れない
# car_1w_pv_flg, car_1m_pv_flg, car_3m_pv_flg, car_6m_pv_flg, car_12m_pv_flg 

#セグメント分布(アクセス時間帯＋休日、平日とか)
# sp_holiday_flg
# sp_weekday_flg
# pc_holiday_flg
# pc_weekday_flg
# holiday_flg
# weekday_flg
# sp_holiday_time_seg
# sp_weekday_time_seg
# pc_holiday_time_seg
# pc_weekday_time_seg
# holiday_time_seg
# weekday_time_seg
# weekday_device_seg

#年収、年齢等の数値変数系は平均と中央値を入れる
# annual_income_predict_value, annual_income_integration_cd,  age_integration_value,

# 市区町村（そもそも埋まってる割合がやや低めなので、難しそうだけど）
# city_integration_nm_1 ~ 3は一回見てみる必要がある
# →精度がえらい低い（27%）なので使わない方がいいかも

#cus_dbのよさ
# →収入がより細かく見れるようになる
# →幼児、小学生以下の子供とか、子供系がもう少し細かく見れる
#Rのサービスユーザ、というバイアスをかけて対象を観察できる
#セグメント分布を見れるようになる

In [18]:
def summarize_cusdb():
    cus = pd.read_csv("/Users/01017387/Desktop/しごと/data/car用/cus_db.csv")
    # 予測含め、地域が入ってないものを捨てる
    # 予測精度約７割程度はあるので、まあいいかなと
    city_drop_na = cus.dropna(subset=['pref_integration_cd'])
    #実際のデータセットはpref_integration_nmで作成する
    
    
    tmp_dt = city_drop_na.ix[:,["count","have_child_integration_flg",
                                "pref_integration_nm"]].groupby(["pref_integration_nm","have_child_integration_flg"]).sum()
    #県別にフラグの値をカウント
    merged = tmp_dt.reset_index().pivot(index=tmp_dt.index.names[0], columns=tmp_dt.index.names[1], values="count")
    lis = []
    for i in merged.columns:
        lis.append(str(merged.columns.name)+"_"+str(i))
    merged.columns = lis
    del merged.index.name
    merged["pref"] = merged.index
    # merged.head()

    #変数を動的に与えて、ループでデータセットを作成＆append
    #フラグ系対象変数
    #sp系は数値が入っていないので、最初は抜いて試す
    # iter_target = ["sex_integration", "age_integration_cd", "high_income_predict_flg",
    #                "businessperson_integration_flg", "couple_flg", "have_baby_flg", "car_1w_pv_flg", "car_1m_pv_flg", "car_3m_pv_flg",
    #                "car_6m_pv_flg", "car_12m_pv_flg", "sp_holiday_flg", "sp_weekday_flg", "pc_holiday_flg", "pc_weekday_flg", "holiday_flg",
    #                "weekday_flg", "sp_holiday_time_seg", "sp_weekday_time_seg", "pc_holiday_time_seg", "pc_weekday_time_seg",
    #                "holiday_time_seg", "weekday_time_seg", "weekday_device_seg"]
    iter_target = ["sex_integration", "age_integration_cd", "high_income_predict_flg",
                   "businessperson_integration_flg", "couple_flg", "have_baby_flg", "car_1w_pv_flg", "car_1m_pv_flg", "car_3m_pv_flg",
                   "car_6m_pv_flg", "car_12m_pv_flg"]

    loop_count = 0
    for i in iter_target:
        loop_count +=1
        tmp_dt = city_drop_na.ix[:,["count",str(i),"pref_integration_nm"]].groupby(["pref_integration_nm",str(i)]).sum()
#         exec('tmp_dt = city_drop_na.ix[:,["count","'+str(i)+'","pref_integration_nm"]].groupby(["pref_integration_nm","'+str(i)+'"]).sum()')
        #県別にフラグの値をカウント
        tmp = tmp_dt.reset_index().pivot(index=tmp_dt.index.names[0], columns=tmp_dt.index.names[1], values="count")
        tmp_rate = tmp.copy()/len(city_drop_na)
        #一時対応策：カラム名が同一になってしまい、後で不都合が起こるので、一時的に適当なカラム名を与える
        tmp_lis = (np.array(list(range(len(tmp_rate.T))))+1)*10
        tmp_rate.columns=tmp_lis

        lis = []
        for n in range(2):
            for i in tmp.columns:
                if n == 0:
                    lis.append(str(tmp.columns.name)+"_"+str(i))
                if n == 1:
                    lis.append(str(tmp.columns.name)+"_"+str(i)+"_rate")

        tmp = pd.concat([tmp,tmp_rate],axis=1)
        tmp.columns = lis
        del tmp.index.name
        tmp["pref"] = tmp.index
        #率も入れる

        #できたものを元のデータセットにマージしていく
        merged = pd.merge(merged,tmp,on="pref")
    #     if loop_count >=11:
    #         break
    
    #数値系の統合(年収、年齢：平均、中央値)
    num_loop = ["annual_income_predict_value", "annual_income_integration_cd", "age_integration_value"]

    for l in num_loop:
        tmp_mean = city_drop_na.ix[:,[str(l),"pref_integration_nm"]].groupby(["pref_integration_nm"]).mean()
        tmp_med = city_drop_na.ix[:,[str(l),"pref_integration_nm"]].groupby(["pref_integration_nm"]).median()
        tmp_mean.columns = [str(l)+"_mean"]
        tmp_med.columns  = [str(l)+"_med"]
#         exec('tmp_mean = city_drop_na.ix[:,["'+str(l)+'","pref_integration_nm"]].groupby(["pref_integration_nm"]).mean()')
#         exec('tmp_med = city_drop_na.ix[:,["'+str(l)+'","pref_integration_nm"]].groupby(["pref_integration_nm"]).median()')
#         exec('tmp_mean.columns = ["'+str(l)+'"+"_mean"]')
#         exec('tmp_med.columns  = ["'+str(l)+'"+"_med"]')
        tmp_mean["pref"] = tmp_mean.index
        tmp_med["pref"] = tmp_med.index
        merged = pd.merge(merged,tmp_mean,on="pref")
        merged = pd.merge(merged,tmp_med,on="pref")

        cus_extracted = merged.copy()
        cus_extracted = cus_extracted.rename(columns={"pref":"chiho"})

        return cus_extracted

In [21]:
#最後に市区町村単位にサマッたカスタマDBを民力にマージする
koku_hoyu_area_cusdb = pd.merge(koku_hoyu_area,cus_extracted,on="chiho")
koku_hoyu_area_cusdb.head()

,pref_code,pref_city_code,capital_city_flg,city_categories,chiho,city_name,population,h22_kumikae_population,population_changed_from_h22_to_h27,rate_population_changed_from_h22_to_h27,...,car_12m_pv_flg_0,car_12m_pv_flg_1,car_12m_pv_flg_0_rate,car_12m_pv_flg_1_rate,annual_income_predict_value_mean,annual_income_predict_value_med,annual_income_integration_cd_mean,annual_income_integration_cd_med,age_integration_value_mean,age_integration_value_med
0,1.0,1100.0,1.0,1.0,北海道,札幌市,"1,952,356","1,913,545","38,811",2.0,...,3456,204,0.045354,0.002677,3.189625,2.6372,2.717213,2,40.027869,38.0
1,1.0,1101.0,NaN,0.0,北海道,札幌市中央区,"237,627","220,189","17,438",7.9,...,3456,204,0.045354,0.002677,3.189625,2.6372,2.717213,2,40.027869,38.0
2,1.0,1102.0,NaN,0.0,北海道,札幌市北区,"285,321","278,781","6,540",2.3,...,3456,204,0.045354,0.002677,3.189625,2.6372,2.717213,2,40.027869,38.0
3,1.0,1103.0,NaN,0.0,北海道,札幌市東区,"261,912","255,873","6,039",2.4,...,3456,204,0.045354,0.002677,3.189625,2.6372,2.717213,2,40.027869,38.0
4,1.0,1104.0,NaN,0.0,北海道,札幌市白石区,"209,584","204,259","5,325",2.6,...,3456,204,0.045354,0.002677,3.189625,2.6372,2.717213,2,40.027869,38.0


<b>------------------------------カスタマDB作成ーここまで---------------------------<br>
------------------------------市区町村名と対応した辞書作成作成ー開始---------------------------<br>
<b/>

<b>
------------------------------市区町村名と対応した辞書作成作成ー終わり---------------------------<br>
<b/>

,wns_bukken_cd,shinkoyo_nohimbi_cd,hojin_cd,madoguchi_cd,chiho,coupon_flg,fair_flg,sd_flg,kuchikomi_flg,maker_name,...,y_cs,p_cs,y_other,p_other,kuchikomi_cnt,katashiki,cnt_total_price,avg_total_price,min_total_price,predicted_price
810772,U00012656926,150102,303968,1,福岡県,1,0,0,1,ダイハツ,...,0,0.037452,0,0.148028,4,LA-L152S,445,3.382966e+05,80000.0,3.140288e+05
5301564,UZ0014935617,150403,219007,1,山梨県,0,0,0,1,ホンダ,...,0,0.074008,0,0.120010,4,DBA-KB2,1,3.212000e+06,3212000.0,2.410772e+06
4924075,UZ0013173022,141107,51721,115,宮城県,1,1,0,1,スバル,...,0,0.026905,0,0.124821,16,DBA-RJ2,156,1.082494e+06,328000.0,1.041860e+06
4500543,UD0015293572,150703,81727,5,滋賀県,0,0,0,0,スズキ,...,0,0.011554,0,0.144302,0,DBA-MA15S,72,1.242056e+06,739000.0,9.695125e+05
4635857,UJ0013369648,141003,30121,1,愛知県,1,1,0,1,三菱,...,0,0.032983,0,0.286805,54,GBD-U61V,1001,7.178082e+05,150000.0,4.494569e+05


<b>
------------------------------データ読み込みステップー終わり---------------------------<br>
------------------------------マージステップー開始---------------------------<br>
<b/>

In [ ]:
#地域

#全体：国勢調査、保有台数、問い合わせ、カスタマDB
# kokusei
# hoyu
# →(koku_hoyu_area)すでに存在する
koku_hoyu_area_cusdb = pd.merge(koku_hoyu_area,cus_extracted,on="chiho")
# koku_hoyu_area_cusdb

In [30]:
#車種
#全体：問い合わせ、お気に入り、カタログ
#車種・グレード別集計（問い合わせ）
shashu_summary_dicts
kuruma["num_shashu"]
# -車種&グレードごと
shashu_grade_summary_dicts
kuruma["num_shashu_grade"]
# grade_by_shashu = convert_to_unit_based_num(shashu_grade_summary,"a.keisai_shashu_kj","a.keisai_grade_name")
#body_typeごと
body_summary_dicts
kuruma["num_body_type"]
#おまけ：ドア数単位
door_summary_dicts
kuruma["num_door"]

#お気に入り
zen #全体
#部分は処理時間がかかりすぎるからダメ

# -車種ごと
kuruma
# カタログ
a12

# →生データとカタログをshashu,FMC,MC,grade_cd,katashikiで紐付ける
raw_cata = raw.merge(a12,on=["shashu_cd","fmc_cd","mc_cd","grade_cd","katashiki"])
#     →上記とお気に入り全体(zen)をshashu,grade_cdで紐付ける
raw_cata_fav = raw_cata.merge(zen,on=["shashu","grade_cd"])
#     →上記と地域データを紐付ける（生データの市区町村名）
cars_cities = raw_cata_fav.merge(koku_hoyu_area_cusdb,on=["city_name"])
# 　→問い合わせ（kuruma）を車種名、型式、市区町村名で紐付ける
for_models = cars_cities.merge(kuruma,on=["shashu","katashiki","city_name"])
#余力があれば期間ごとに集計したお気に入り件数をマージする

In [ ]:
#全体
mrd = make_raw_data()
#成約データの読み込み
mrd.make_seiyaku()
#地域情報の作成
mrd.make_kokusei()
mrd.make_hoyu()
mrd.make_toiawase()
mrd.make_cus_db()
#車両情報の作成
mrd.make_favorites()
mrd.make_catalogue()
#マージ
mrd.merge_all()
